In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

# Parameters for road surface generation
m_road = 2  # Pavement Waviness Indicator
N = 1000  # Number of samples
omega0 = 1  # Spatial Reference Frequency
phi_omega0 = 4E-6  # PSD value at the reference frequency
lower_bound = 2 * np.pi * 0.1  # Lower limit of frequency sampling
upper_bound = 2 * np.pi * 10  # Upper limit of frequency sampling
v = 90 * 1000 / 3600  # Speed in m/s

# Generate the road surface
omega_vect = np.linspace(lower_bound, upper_bound, N)  # Frequencies to sample
phi_vect = phi_omega0 * (omega_vect / omega0) ** -m_road
delta_omega = (omega_vect[-1] - omega_vect[0]) / (N - 1)
amp_vect = np.sqrt(phi_vect * (delta_omega / np.pi))

t_vals = np.linspace(0, 100 / v, 1000)  # Time values based on speed and spatial range
road_surface = np.zeros_like(t_vals)

for ii in range(N):
    phase = 2 * np.pi * np.random.rand()  # Random phase shift
    road_surface += amp_vect[ii] * np.sin(omega_vect[ii] * v * t_vals - phase)

# Interpolate the road surface
f_interp = interp1d(t_vals, road_surface, kind='cubic', fill_value="extrapolate")

# Compute the derivative f'(t) numerically
f_prime_vals = np.gradient(road_surface, t_vals)
f_prime_interp = interp1d(t_vals, f_prime_vals, kind='cubic', fill_value="extrapolate")

# Define the forcing function f(t) = (k/m)f + (c/m)f'
def forcing_function(t, k, m, c):
    return (k / m) * f_interp(t) + (c / m) * f_prime_interp(t)

# System parameters
m = 100  # Mass of the trailer system
c = 0.1  # Trailer damping coefficient
k = 100  # Trailer spring constant

# Define the system of equations
def second_order_ode(t, y, p, q, k, m, c):
    y1, y2 = y  # y1 = y, y2 = y'
    f_t = forcing_function(t, k, m, c)  # Compute the forcing function
    dydt = [y2, -p * y2 - q * y1 + f_t]  # [y', y'']
    return dydt

# Define parameters
p = c / m
q = k / m

# Initial conditions: y(0) = 1, y'(0) = 0
y0 = [1, 0]

# Solve the ODE
t_span = (0, 10)
t_eval = np.linspace(0, 10, 100)
solution = solve_ivp(second_order_ode, t_span, y0, t_eval=t_eval, args=(p, q, k, m, c))

# Extract y(t) and y'(t)
y = solution.y[0]       # y(t)
y_dot = solution.y[1]   # y'(t)
t = solution.t          # Time points

# Compute the second derivative (acceleration)
acceleration = -p * y_dot - q * y + forcing_function(t, k, m, c)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(t, y, label="y(t) (Displacement)")
plt.plot(t, y_dot, label="y'(t) (Velocity)")
plt.plot(t, acceleration, label="y''(t) (Acceleration)", linestyle="--")
plt.xlabel("Time t (s)")
plt.ylabel("Response")
plt.legend()
plt.grid()
plt.title("System Response: Displacement, Velocity, and Acceleration")
plt.show()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\miste\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\miste\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\miste\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\miste\AppData\Roaming\Python\Python310\site-packages\traitlets\config\applicatio

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import